<a href="https://colab.research.google.com/gist/jirislav/173dc2ba48959753a124b11f1719bafa/apache-spark-in-practice-it-academy-2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What you'll put your hands on in this notebook

- reading data from the Parquet format
- linking two separate data sets together based on a common field
- writing simple aggregation

# Preparing the data & Spark session

Following snippet will download our example dataset which you'll be working with:

In [25]:
!test -f example-dataset.tar.xz || wget https://github.com/seznam/IT-akademie-bigdata/raw/main/big-data/data/example-dataset.tar.xz
!test -d example-dataset || tar -xf example-dataset.tar.xz
!ls -l

--2022-05-16 07:00:41--  https://github.com/seznam/IT-akademie-bigdata/raw/main/big-data/data/example-dataset.tar.xz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/seznam/IT-akademie-bigdata/main/big-data/data/example-dataset.tar.xz [following]
--2022-05-16 07:00:42--  https://raw.githubusercontent.com/seznam/IT-akademie-bigdata/main/big-data/data/example-dataset.tar.xz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7190296 (6.9M) [application/octet-stream]
Saving to: ‘example-dataset.tar.xz’

example-dataset.tar 100%[===================>]   6.86M  --.-KB/s    in 0.04s   

2022-05-16 07:00:42 

Now let's install Spark on PySpark:

In [10]:
# Install Spark

import os
os.chdir("/content")
!test -f spark-3.2.1-bin-hadoop2.7.tgz || wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!test -d spark-3.2.1-bin-hadoop2.7 || tar -xf spark-3.2.1-bin-hadoop2.7.tgz

# Setup pyspark
!pip install findspark
import findspark
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
findspark.init()

# Create new SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

--2022-05-16 06:12:41--  https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M  7.06MB/s    in 47s     

2022-05-16 06:13:28 (5.48 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [16]:
# Just some handy functions to keep the code cells clean later on
# Note that we don't use asterisk (*) because then the Colab completion doesn't work
from pyspark.sql.functions import col, floor, udf, explode

# Reading the data

Now it's time for you to come up with what you've learned from the previous [Introduction to Apache Spark - IT Academy 2022](https://colab.research.google.com/github/seznam/IT-akademie-bigdata/blob/main/big-data/notebooks/001_introduction_to_apache_spark.ipynb) notebook.

Let us help you for starters by looking into the structure of the data we have prepared for you.

In [7]:
!ls -l example-dataset/

total 8
drwxrwxr-x 2 1000 1000 4096 May  3 16:02 clicks
drwxrwxr-x 2 1000 1000 4096 May  3 16:04 impressions


Okay, we have two directories `clicks` and `impressions`. There were taken from Seznam's ad division, so a click means some user clicked an ad, while an impression means an ad was loaded by the browser and rendered.

Let's look into the directories to see the data format:

In [8]:
!ls -l example-dataset/*

example-dataset/clicks:
total 5376
-rw-rw-r-- 1 1000 1000 4065411 May  3 16:03 part-00000-fb446a88-0486-413b-87b9-e9c9f930d3ab-c000.snappy.parquet
-rw-rw-r-- 1 1000 1000 1435727 May  3 16:03 part-00001-fb446a88-0486-413b-87b9-e9c9f930d3ab-c000.snappy.parquet
-rw-rw-r-- 1 1000 1000       0 May  3 16:03 _SUCCESS

example-dataset/impressions:
total 4416
-rw-rw-r-- 1 1000 1000 1130364 May  3 16:04 part-00000-456bf299-6efb-45f1-905e-2512ed65d1fe-c000.snappy.parquet
-rw-rw-r-- 1 1000 1000 1126469 May  3 16:04 part-00001-456bf299-6efb-45f1-905e-2512ed65d1fe-c000.snappy.parquet
-rw-rw-r-- 1 1000 1000 1126712 May  3 16:04 part-00002-456bf299-6efb-45f1-905e-2512ed65d1fe-c000.snappy.parquet
-rw-rw-r-- 1 1000 1000 1128897 May  3 16:04 part-00003-456bf299-6efb-45f1-905e-2512ed65d1fe-c000.snappy.parquet
-rw-rw-r-- 1 1000 1000       0 May  3 16:04 _SUCCESS


As you can see, the data are using Parquet format. And here comes your first task:
- load clicks parquet directory into one DataFrame (named `clicks`)
- and impressions directory into other DataFrame (named `impressions`)

*HINT: [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameReader.parquet.html?highlight=parquet) might be handy if autocompletion does not suffice*

In [12]:
clicks = 

In [13]:
impressions = 

Let's see the schema and the data to verify you have loaded it successfully

In [14]:
clicks.printSchema()
clicks.show()

root
 |-- Timestamp: long (nullable = true)
 |-- ImpressionTimestamp: long (nullable = true)
 |-- AdId: integer (nullable = true)
 |-- KeywordId: long (nullable = true)
 |-- RandomId: long (nullable = true)

+----------------+-------------------+---------+----------+----------+
|       Timestamp|ImpressionTimestamp|     AdId| KeywordId|  RandomId|
+----------------+-------------------+---------+----------+----------+
|1649307603675276|   1649307595649634|435947651|      null|2062901616|
|1649307603734501|   1649307585109683|554794504|      null|3257733838|
|1649307604043892|   1649307589191166|575725918|      null|2184303841|
|1649307604149413|   1649307559129589|575725918|      null|1839752057|
|1649307606070591|   1649306613659472|571896132|      null|3358643236|
|1649307607675483|   1649307604482729|     null|      null|1556358807|
|1649307608554653|   1649307563750753|575351159|      null|3988672169|
|1649307609072961|   1649307559897859|573836983|      null|1161004805|
|1649307609

In [15]:
impressions.printSchema()
impressions.show()

root
 |-- bidTimestamp: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- requestId: string (nullable = true)
 |-- randomIds: array (nullable = true)
 |    |-- element: long (containsNull = true)

+----------------+----------------+--------------------+--------------------+
|    bidTimestamp|       timestamp|           requestId|           randomIds|
+----------------+----------------+--------------------+--------------------+
|1649282455048002|1649282455048002|347f3b79-57d9-4b3...|        [3783526641]|
|1649282808972936|1649282808972936|3de24cdf-d2a6-4d1...|[1352518872, 1036...|
|1649282887994142|1649282887994142|dfbb298c-239c-4ae...|         [990130250]|
|1649282987862427|1649282987862427|3e518073-1612-4b5...|        [4055324047]|
|1649283009547800|1649283009547800|8b3bc895-03a1-4f7...|        [1267857419]|
|1649283101173985|1649283101173985|8ccfa691-c760-47e...|        [1544212150]|
|1649283335899174|1649283335899174|bfa1a0e8-bb15-465...|        [2693591608]|
|16492

Great! At this point, you're ready to go on to the second part.

# Linking two datasets together

As you can see from the schemas of both DataFrames, there are some fields, which we can use to link the data together.

One of them is `click.ImpressionTimestamp` column, which should correlate to `impression.timestamp` column. We will use this to link the two datasets together.

The second one is a bit tricky to understand, because first you need to understand, what is an impression within our dataset.
- single line of `impressions` DataFrame represents a single rendering result from the browser, but it potentially includes more ads at once, which is why there is an column named `randomIds`, which is actually an array of integers.
- each `randomId` can be considered as a single ad
- since one line of `clicks` DataFrame represents a single click on a single ad, we can link `impressions` to `clicks` only after we *explode* our `randomIds` array
  - what do we mean by *exploding* the array?
  - well, it's like flattening the structure, so that we get "more lines" in the DataFrame at the end, so that we have one impressions with exactly one randomId

That said, we now know we can also link `click.RandomId` field with exploded `impression.randomIds[]` array.

See function docs you should use:
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode.html

In [17]:
exploded_impressions = 

In [18]:
# Before we proceed to joining two data frames, we need to unify correlated column names
exploded_impressions = exploded_impressions \
                        .drop('randomIds') \
                        .withColumnRenamed('timestamp', 'impressionTimestamp')

Let's see what we've got:

In [21]:
exploded_impressions.printSchema()
exploded_impressions.show()

root
 |-- bidTimestamp: long (nullable = true)
 |-- impressionTimestamp: long (nullable = true)
 |-- requestId: string (nullable = true)
 |-- randomId: long (nullable = true)

+----------------+-------------------+--------------------+----------+
|    bidTimestamp|impressionTimestamp|           requestId|  randomId|
+----------------+-------------------+--------------------+----------+
|1649282455048002|   1649282455048002|347f3b79-57d9-4b3...|3783526641|
|1649282808972936|   1649282808972936|3de24cdf-d2a6-4d1...|1352518872|
|1649282808972936|   1649282808972936|3de24cdf-d2a6-4d1...|1036945266|
|1649282887994142|   1649282887994142|dfbb298c-239c-4ae...| 990130250|
|1649282987862427|   1649282987862427|3e518073-1612-4b5...|4055324047|
|1649283009547800|   1649283009547800|8b3bc895-03a1-4f7...|1267857419|
|1649283101173985|   1649283101173985|8ccfa691-c760-47e...|1544212150|
|1649283335899174|   1649283335899174|bfa1a0e8-bb15-465...|2693591608|
|1649283409904003|   1649283409904003|53da9

Now that we've exploded impressions using `randomIds`, it's time for you to join these DataFrames by `randomId` and  `impressionTimestamp` fields. Please consult the docs:
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.join.html

In [19]:
linked_impressions = 

In [20]:
# To avoid ambiguity, rename Timestamp column from clicks
linked_impressions = linked_impressions \
                      .withColumnRenamed('Timestamp', 'ClickTimestamp') 

Let's see if we got it right:

In [22]:
linked_impressions.printSchema()
linked_impressions.show()

root
 |-- ImpressionTimestamp: long (nullable = true)
 |-- RandomId: long (nullable = true)
 |-- ClickTimestamp: long (nullable = true)
 |-- AdId: integer (nullable = true)
 |-- KeywordId: long (nullable = true)
 |-- bidTimestamp: long (nullable = true)
 |-- requestId: string (nullable = true)

+-------------------+----------+----------------+---------+---------+----------------+--------------------+
|ImpressionTimestamp|  RandomId|  ClickTimestamp|     AdId|KeywordId|    bidTimestamp|           requestId|
+-------------------+----------+----------------+---------+---------+----------------+--------------------+
|   1649308746783112|2697130188|1649308755993175|575498235|     null|1649308746783112|1ec695a2-2c1e-4eb...|
|   1649308025299273|3349023642|1649308043306380|575424121|     null|1649308025299273|5383ca69-0a90-4d9...|
|   1649308630214863|3010845523|1649308646358244|575424120|     null|1649308630214863|4ebbab4b-c4a8-49b...|
|   1649308629834350|3704989181|1649308658448623|5754241

## Bonus task

- create also unlinked DataFrame, which will contain all these impressions or clicks, which were not linked by the other DataFrame

In [24]:
# Hint: using just "DataFrame.join" method is enough

unlinked_impressions =

unlinked_impressions.printSchema()
unlinked_impressions.count()

root
 |-- randomId: long (nullable = true)
 |-- impressionTimestamp: long (nullable = true)
 |-- bidTimestamp: long (nullable = true)
 |-- requestId: string (nullable = true)
 |-- ClickTimestamp: long (nullable = true)
 |-- AdId: integer (nullable = true)
 |-- KeywordId: long (nullable = true)
 |-- bidTimestamp: long (nullable = true)
 |-- requestId: string (nullable = true)



81702

# Performing basic aggregation

Your task here is to create a summary statistics of how many clicks were registered per each `AdId`.

For example:

| AdId | Clicks |
| - | - |
| 1002 | 30 |
| 586 | 2 |
| ... | ... |

*Hint: Use group by aggregation from the [Introduction to Apache Spark - IT Academy 2022](https://colab.research.google.com/github/seznam/IT-akademie-bigdata/blob/main/big-data/notebooks/001_introduction_to_apache_spark.ipynb) notebook.*

In [ ]:
clicks_by_ad_id =

clicks_by_ad_id.printSchema()
clicks_by_ad_id.show()


And that's about it!